In [16]:
import requests
import pandas as pd
import time
import os

In [2]:

def get_token(username,password,authsrc):
    
    url = "https://vrom1.webex.com/suite-api/api/auth/token/acquire"

    payload = "{\n\t\"username\" : \""+username+"\",\n\t\"authSource\" : \""+authsrc+"\",\n\t\"password\" : \""+password+"\"\n}"
    headers = {
      'Accept': 'application/json',
      'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data = payload,verify=False)
    r = dict(response.json())
    return r["token"]



In [3]:
def get_cluster_id(token):
    url = "https://vrom1.webex.com/suite-api/api/resources?resourceKind=ClusterComputeResource"

    payload = {}
    headers = {
      'Accept': 'application/json',
      'Authorization': 'vRealizeOpsToken '+token
    }

    response = requests.request("GET", url, headers=headers, data = payload,verify=False)
    r = dict(response.json())
    
    resourceId = dict()
    for cluster in r["resourceList"]:
        resourceId[cluster['resourceKey']['name']]=cluster["identifier"]
    return resourceId

In [4]:
def get_stat_keys(token,resourceId):
    url = "https://vrom1.webex.com/suite-api/api/resources/"+resourceId+"/statkeys"

    payload = {}
    headers = {
      'Accept': 'application/json',
      'Authorization': 'vRealizeOpsToken '+token
    }

    response = requests.request("GET", url, headers=headers, data = payload,verify=False)
    r = dict(response.json())
    
    stat_keys = []
    for d in r["stat-key"]:
        stat_keys.append(d["key"])
    return stat_keys

In [59]:
def get_stat_value(token,resourceId,days):
    
    end =  int(round(time.time() * 1000))
    start = end - 24*60*60*1000*days
    
    url = "https://vrom1.webex.com/suite-api/api/resources/stats/query"
    
    payload = "{\n\t\"begin\":"+str(start)+",\n\t\"end\":"+str(end)+",\n\t\"intervalType\":\"DAYS\",\n\t\"intervalQuantifier\":1,\n\t\"rollUpType\":\"AVG\",\n\t\"resourceId\":[\n\t\t\""+resourceId+"\"\n\t],\n\t\"statKey\": [\n\t\t\"mem|usage_average\",\"cpu|usagemhz_average\",\"diskspace|total_usage\"\n\t]\n}"
    headers = {
      'Accept': 'application/json',
      'Authorization': 'vRealizeOpsToken '+token,
      'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data = payload,verify=False)
    r = dict(response.json())
    if len(r["values"][0]["stat-list"]['stat'])<3:
        return []
    timestamps  = []
    timestamps1 = r["values"][0]["stat-list"]['stat'][0]['timestamps']
    timestamps2 = r["values"][0]["stat-list"]['stat'][1]['timestamps']
    timestamps3 = r["values"][0]["stat-list"]['stat'][2]['timestamps']
    print(len(timestamps1),len(timestamps2),len(timestamps3))
    if len(timestamps)<len(timestamps1):
        timestamps = timestamps1
    if len(timestamps)<len(timestamps2):
        timestamps = timestamps2
    if len(timestamps)<len(timestamps3):
        timestamps = timestamps3
    
    mem  = (len(timestamps)-len(timestamps1))*[0]+r["values"][0]["stat-list"]['stat'][0]['data']
    cpu  = (len(timestamps)-len(timestamps2))*[0]+r["values"][0]["stat-list"]['stat'][1]['data']
    disk = (len(timestamps)-len(timestamps3))*[0]+r["values"][0]["stat-list"]['stat'][2]['data']
    print(len(mem),len(cpu),len(disk))
    df = pd.DataFrame()
    
    df["timestamps"] = timestamps
    df["mem"] = mem
    df["cpu"] = cpu
    df["disk"] = disk
    
    return df
    

In [7]:
token = get_token("iaas-india-dev.gen","Vrom@1234","Fusion-AD-LDAP")

/Users/abhawast/python3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vrom1.webex.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [8]:
resourceId = get_cluster_id(token)

/Users/abhawast/python3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vrom1.webex.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [27]:
stat_keys = get_stat_keys(token,resourceId['AMS01-TVPP1-VMR-Cluster'])

/Users/abhawast/python3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vrom1.webex.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [56]:
df = get_stat_value(token,resourceId['SJC02-TVPP5-Unallocated'],210)

/Users/abhawast/python3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vrom1.webex.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [11]:
len(df)

211

In [12]:
df.head()

timestamps        mem           cpu         disk
0  1569887999999  20.809977  61335.775000  4085.305713
1  1569974399999  20.820082  61442.117540  4085.297674
2  1570060799999  20.836326  61363.786092  4085.288567
3  1570147199999  20.846651  61508.581733  4085.285675
4  1570233599999  20.863940  61882.738390  4085.283981

In [62]:
clusters = list(resourceId.keys())
def store_cluster_to_csv(clusters):
    for cluster in clusters:
        df = get_stat_value(token,resourceId[cluster],210)
        try:
            os.remove(cluster+".csv")
        except:
            print(cluster+" doesnt exist")
        if len(df)!=0:
            df.to_csv(cluster+".csv")
            print(cluster+" written")


In [64]:
store_cluster_to_csv(clusters)

In [18]:
clusters

['AMS01-TVPP1-VMR-Cluster',
 'AMS01-VPP1-GSB-Miscellaneous-Cluster',
 'SJC02-TVPP1-Unallocated',
 'DFW02-TVPP2-VTS-Cluster',
 'SJC02-VPP7-Management-Cluster',
 'SIN01-TVPP1-vTS-M4-Cluster',
 'DFW01-VPP2-Management-Cluster',
 'SJC02-VPP2-Meeting-NBRWES',
 'DFW02-VPP1-Meeting-NBRWES2',
 'SJC02-TVPP5-vTS-Cluster',
 'SYD01-VPP1-Infrastructure',
 'AMS01-TVPP1-vTS-M4-Cluster',
 'DFW02-VPP1-Meeting2-Cluster',
 'NSX-POC',
 'SYD10-IPOP1-Meeting2-Cluster',
 'SJC02-VPP2-TPGW-Cluster',
 'DFW01-VPP2-SDE-Cluster',
 'DFW01-VPP2-Jabber-Cluster',
 'SIN01-VPP1-NBRWES2-Cluster',
 'SIN01-TVPP1-Management-Cluster',
 'SJC02-VPP7-Acano-Cluster',
 'SJC02-DC3-BTS-WebEx11-Cluster',
 'LHR03-TVPP1-CloudCenter-Cluster2',
 'LHR03-VPP1-WebEx11-Cluster',
 'SJC02-VPP2-Meeting-Cluster',
 'DFW01-VPP1-Management-Cluster',
 'SJC02-TVPP5-Unallocated',
 'JFK01-VPP1-MMP2',
 'IAD02-VPP1-Meeting-Cluster',
 'SJC02-TVPP1-vTSPOC-Cluster',
 'SJC02-VPP2-WebEx11-Cluster',
 'LHR03-TVPP1-Tahoe-CMS',
 'SJC02-VPP2-Miscellaneous-Cluster'